In [ ]:
!pip install langchain google-cloud-aiplatform google-auth > /dev/null
!pip install arxiv > /dev/null
!pip install duckduckgo-search > /dev/null
!pip install googlemaps > /dev/null
!pip install gradio_tools > /dev/null
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install Pydrive2

In [ ]:
from langchain.document_loaders import WhatsAppChatLoader

In [49]:
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.agents import load_tools, initialize_agent, AgentType

In [48]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='generativeaitrial-trialLC.json'

In [31]:
from langchain.document_loaders import JSONLoader

In [33]:
import json
from pathlib import Path
from pprint import pprint


file_path='trial_proposals.json'
data = json.loads(Path(file_path).read_text())

In [34]:
pprint(data)

[{'authors': [],
  'champions': ['Domenic Denicola'],
  'description': 'TC39 meeting notes',
  'has-specification': False,
  'name': 'Normative ICU Reference',
  'pushed_at': '2023-06-08T21:29:18.000Z',
  'rationale': 'Withdrawn; champion is no longer participating in TC39',
  'stage': -1,
  'tags': ['ECMA-262', 'withdrawn'],
  'url': 'https://github.com/tc39/notes'},
 {'authors': ['Philipp Dunkel',
              'Maggie Johnson-Pint',
              'Matt Johnson-Pint',
              'Brian Terlson',
              'Shane Carr',
              'Ujjwal Sharma',
              'Philip Chimento',
              'Jason Williams',
              'Justin Grant'],
  'champions': ['Philipp Dunkel',
                'Maggie Johnson-Pint',
                'Matt Johnson-Pint',
                'Brian Terlson',
                'Shane Carr',
                'Ujjwal Sharma',
                'Philip Chimento',
                'Jason Williams',
                'Justin Grant'],
  'description': 'Provides stan

  'name': 'Deep Path Properties in Record Literals',
  'notes': [{'date': '2020-06-03T00:00:00.000Z',
             'url': 'https://github.com/tc39/notes/blob/HEAD/meetings/2020-06/june-3.md#deep-path-properties'}],
  'pushed_at': '2020-06-03T17:35:41.000Z',
  'stage': 1,
  'tags': ['ECMA-262'],
  'url': 'https://github.com/tc39/proposal-deep-path-properties-for-record'},
 {'authors': [],
  'champions': ['Hemanth HM', 'Jordan Harband'],
  'description': 'Proposal Generic Comparison',
  'has-specification': False,
  'id': 'proposal-generic-comparison',
  'name': 'Generic Comparison',
  'pushed_at': '2020-05-29T04:12:01.000Z',
  'rationale': 'Withdrawn: replaced with Array Equality',
  'stage': -1,
  'tags': ['ECMA-262', 'withdrawn'],
  'url': 'https://github.com/hemanth/proposal-generic-comparison'},
 {'authors': ['Domenic Denicola'],
  'champions': ['Gus Caplan'],
  'description': 'import.meta proposal for JavaScript',
  'edition': 2020,
  'has-specification': True,
  'id': 'proposal-im

In [36]:
loader = JSONLoader(
    file_path='trial_proposals.json',
    jq_schema='.',
text_content=False)

data = loader.load()

In [37]:
data

[Document(page_content='[{\'tags\': [\'ECMA-262\', \'withdrawn\'], \'stage\': -1, \'name\': \'Normative ICU Reference\', \'description\': \'TC39 meeting notes\', \'url\': \'https://github.com/tc39/notes\', \'has-specification\': False, \'authors\': [], \'champions\': [\'Domenic Denicola\'], \'rationale\': \'Withdrawn; champion is no longer participating in TC39\', \'pushed_at\': \'2023-06-08T21:29:18.000Z\'}, {\'tags\': [\'ECMA-262\'], \'stage\': 3, \'name\': \'Temporal\', \'id\': \'proposal-temporal\', \'description\': \'Provides standard objects and functions for working with dates and times.\', \'url\': \'https://github.com/tc39/proposal-temporal\', \'notes\': [{\'date\': \'2021-12-14T00:00:00.000Z\', \'url\': \'https://github.com/tc39/notes/blob/HEAD/meetings/2021-12/dec-14.md#temporal-update\'}, {\'date\': \'2017-07-27T00:00:00.000Z\', \'url\': \'https://github.com/tc39/notes/blob/HEAD/meetings/2017-07/jul-27.md\'}, {\'date\': \'2018-09-27T00:00:00.000Z\', \'url\': \'https://githu

In [38]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["authors"] = record.get("authors")
    metadata["champions"] = record.get("champions")
    metadata["description"] = record.get("description")
    metadata["has-specification"] = record.get("has-specification")
    metadata["name"] = record.get("name")
    metadata["pushed_at"] = record.get("pushed_at")
    metadata["rationale"] = record.get("rationale")
    metadata["stage"] = record.get("stage")
    metadata["url"] = record.get("url")
    
    return metadata

In [41]:
loader = JSONLoader(
    file_path='trial_proposals.json',
    jq_schema='.',
    text_content=False,
    metadata_func=metadata_func
)

In [77]:
#the chunks from the loader.load_and_split() will be big

from langchain.text_splitter import RecursiveCharacterTextSplitter

json_split = RecursiveCharacterTextSplitter(chunk_size=800,
                                            chunk_overlap=10,
                                           length_function=len)

In [78]:
data = loader.load_and_split(text_splitter=json_split)

In [79]:
len(data)

240

In [80]:
data[0]

Document(page_content="[{'tags': ['ECMA-262', 'withdrawn'], 'stage': -1, 'name': 'Normative ICU Reference', 'description': 'TC39 meeting notes', 'url': 'https://github.com/tc39/notes', 'has-specification': False, 'authors': [], 'champions': ['Domenic Denicola'], 'rationale': 'Withdrawn; champion is no longer participating in TC39', 'pushed_at': '2023-06-08T21:29:18.000Z'}, {'tags': ['ECMA-262'], 'stage': 3, 'name': 'Temporal', 'id': 'proposal-temporal', 'description': 'Provides standard objects and functions for working with dates and times.', 'url': 'https://github.com/tc39/proposal-temporal', 'notes': [{'date': '2021-12-14T00:00:00.000Z', 'url': 'https://github.com/tc39/notes/blob/HEAD/meetings/2021-12/dec-14.md#temporal-update'}, {'date': '2017-07-27T00:00:00.000Z', 'url':", metadata={'source': '/run/media/solverbot/repoA/gitFolders/python_de_learners_data/code_script_notebooks/projects/langChain_exploration/trial_proposals.json', 'seq_num': 1})

In [82]:
from langchain.vectorstores import FAISS

doc_load = FAISS.from_documents(documents=data[:100],
                               embedding=VertexAIEmbeddings())

In [83]:
doc_load.save_local("proposal_faiss")

In [93]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(VertexAI(temperature=0,top_p=0.5,
                                           top_k=5), 
                                   chain_type="stuff")

def get_reply(query):
    docs = doc_load.similarity_search(query,k=1)
    output = chain({"input_documents": docs, 
           "question": query}, 
          return_only_outputs=True)
    return output

In [94]:
doc_load.similarity_search("How many time 'Domenic Denicola' is the champion",k=1)

[Document(page_content="{'date': '2018-01-24T00:00:00.000Z', 'url': 'https://github.com/tc39/notes/blob/HEAD/meetings/2018-01/jan-24.md'}, {'date': '2020-06-02T00:00:00.000Z', 'url': 'https://github.com/tc39/notes/blob/HEAD/meetings/2020-06/june-2.md'}], 'has-specification': True, 'authors': ['Mathias Bynens'], 'champions': ['Brian Terlson', 'Daniel Ehrenberg', 'Mathias Bynens'], 'edition': 2018, 'pushed_at': '2023-03-20T23:39:15.000Z'}, {'tags': ['ECMA-262'], 'stage': 3, 'name': 'Duplicate named capture groups', 'id': 'proposal-duplicate-named-capturing-groups', 'description': 'TC39 proposal to allow regex capturing group names to be repeated', 'url': 'https://github.com/tc39/proposal-duplicate-named-capturing-groups', 'notes': [{'date': '2022-07-20T00:00:00.000Z', 'url':", metadata={'source': '/run/media/solverbot/repoA/gitFolders/python_de_learners_data/code_script_notebooks/projects/langChain_exploration/trial_proposals.json', 'seq_num': 1})]

In [95]:
get_reply("How many time 'Domenic Denicola' is the champion")

InvalidArgument: 400 5.000000 is out of supported range [0, 1];  for top_p.